<a href="https://colab.research.google.com/github/YuqiYuqiMa/Enhanced-Face-Recognition-for-Surveillance-Systems/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import warnings

drive.mount('/content/drive')
%cd  /content/drive/My\ Drive/CMPE\ 295A

Mounted at /content/drive
/content/drive/My Drive/CMPE 295A


In [2]:
import csv
from collections import defaultdict
import nltk
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

In [9]:
words = []
words_file = open('data/train.clabel.txt', 'r') 
for w in words_file.readlines():
    words.append(w.strip())

In [ ]:
print("total number of documents: ", len(words), "\n")

for i in range(10):
    print(i+1, ": ", words[i])

total number of documents:  126373 

1 :  rapid
2 :  heartbeat
3 :  forc
4 :  thoma
5 :  leav
6 :  game
7 :  star
8 :  expect
9 :  plai
10 :  weekend


In [ ]:
stopw = stopwords.words('english')
stopwords_ids = []

for i in range(len(words)):
    if words[i] in stopw:
        stopwords_ids.append(i)
        
print(stopwords_ids)

[52, 149, 227, 257, 326, 336, 386, 404, 617, 626, 726, 727, 868, 992, 1235, 1284, 1313, 1845, 1864, 2040, 2286, 2512, 2517, 2526, 2673, 2820, 3365, 3663, 3707, 4264, 4330, 4789, 5036, 5922, 5930, 6083, 6193, 7131, 7334, 7685, 8313, 9263, 9273, 11970, 13280, 14088, 14243, 15063, 15273, 16221, 19271, 19337, 19864, 20154, 23535, 25118, 27240, 27560, 28464, 28856, 29590, 32366, 32388, 33061, 34163, 36067, 37202, 37733, 37807, 38993, 44901, 45527, 47702, 49516, 51235, 51671, 51856, 55384, 60556, 79915, 80484, 81766, 82730, 94390, 94392, 95454, 96521, 97791, 100417, 103389, 106530, 109639, 110104, 120590, 120654, 121122, 124422]


In [3]:
train = []
max_id = 0
open_file = open('data/train.dat.txt', 'r') 

lines = open_file.readlines()
nrows = len(lines)
ncols = 0 
nnz = 0 

for line in lines:
    words = line.split(" ")
    nnz += round(len(words)/2)
            
val = np.zeros(nnz, dtype=np.float)
ind = np.zeros(nnz, dtype=np.int)
ptr = np.zeros(nrows+1, dtype=np.long)
n = 0 

for i in range(nrows):
    words = lines[i].split(" ")
    for j in range(0, len(words), 2): 
        word_id = int(words[j])
        count = float(words[j+1])
        ncols = max(ncols, word_id)
        
        ind[n] = word_id - 1
        val[n] = count
        n += 1
    ptr[i+1] = n 

mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.float)
mat.sort_indices()

In [6]:
mat.shape

(8580, 126355)

In [4]:
def find_clusters(mat, centers):
    indices = []
#     print(mat.shape, centers.shape)
    sim = mat.dot(centers.T)
  
    for i in range(sim.shape[0]):
        row = sim.getrow(i).toarray()[0].ravel()
        indices.append(row.argsort()[-1] + 1)
        
    return indices


def get_centers(mat, k, indices):
    centers = []
    for i in range(1, k+1):
        check = [a for a, b in enumerate(indices) if b == i]
        words = mat[check].toarray().mean(0)
#         print(type(words), words)
        if words.shape[0] > 1:
            centers.append(words)
            
    return csr_matrix(centers)


def kmeans(mat, k):
    centers = shuffle(mat, random_state=0)[:k]
    for _ in range(100): 
        indices = find_clusters(mat, centers)
        centers = get_centers(mat, k, indices)        
    return indices


def bisect(mat, k):
    clusters = [1] * mat.shape[0]
    
    for i in range(1, k):
        data = [0] * (mat.shape[0] + 1)
        record = []
        temp = []

        for j in range(mat.shape[0]): 
            if clusters[j] == 1:              
                temp.append(mat[j].toarray()[0])
                record.append(j)
            else:
                clusters[j] += 1
                       
        km_res = kmeans(csr_matrix(temp), k)
        
        for j in range(len(km_res)):
            bi = km_res[j]
            if bi == 2:
                a = record[j]
                clusters[a] = bi      
            
        for j in range(mat.shape[0]):  
            a = clusters[j] - 1;
            data[a] += 1
        
    return clusters


In [ ]:
c = bisect(mat, 7)

In [ ]:
len(c)

In [ ]:
sum(c)/len(c)

In [ ]:
def write_to_output(clusters):
    open_file = open('format.dat.txt', 'w')
    csv_writer = csv.writer(open_file)
    csv_writer.writerow(["DocumentID", "ClusterID"])
    for i in range(len(clusters)):
        csv_writer.writerow([str(i+1), str(int(clusters[i]))])

In [ ]:
write_to_output(c)

In [ ]:
def sse(mat, clusters, k):
    error = 0
    for i in range(1, k+1):
        group = [mat[j] for j in range(mat.shape[0]) if clusters[j] == i]
        center = group.toarray().mean(0)
        e = np.sum(np.linalg.norm(points-centroid, ord=2, axis=1))
        print("k = ", k, ", sse = ", e)
        error += e
    return error

In [ ]:
x = []
y = []

for k in range(3, 21, 2):
    x.append(k)
    clusters = bisect(mat, k)
    y.append(mat, clusters, k)